In [3]:
import numpy as np
from sparse_vector import *
from vector_polynomial_isomorphism import *

We note that we are using the base field `QQ` for all computations currently

In [54]:
# Let us initialize constants and vector spaces.

N = 18
d = 3
indeterminates = [var("".join(["x_", str(i), str(j), str(k)])) for i in range(2) for j in range(3) for k in range(3)]
R = PolynomialRing(QQ, indeterminates)
R.inject_variables()

glN = MatrixSpace(QQ, N)
dimV = binomial(N + d - 1, d)
V = VectorSpace(QQ, dimV)

vp_iso = vector_polynomial_isomorphism(R, V, indeterminates, d)

Defining x_000, x_001, x_002, x_010, x_011, x_012, x_020, x_021, x_022, x_100, x_101, x_102, x_110, x_111, x_112, x_120, x_121, x_122


In [55]:
# construct f

M_6by3 = MatrixSpace(R, 3, 6)

x_sub = lambda i,j,k: R(var("".join(["x_", str(i), str(j), str(k)])))

A_12 = M_6by3(0)
A_13 = M_6by3(0)

for i in range(2):
    for j in range(3):
        for k in range(3):
            A_12[k, 3*i + j] = x_sub(i,j,k)
            A_13[j, 3*i + k] = x_sub(i,j,k)

f = A_12.minors(3) + A_13.minors(3)

f_vec = [vp_iso.polynomial_to_vector(fi) for fi in f]


In [112]:
# define W = span{f}, W_perp = orthogonal complement of span{f}

W = V.subspace(f_vec)

f_vec_basis = W.basis() # take only the linearly independent ones
    
grad_f = [vp_iso.vector_to_polynomial(fi).gradient() for fi in f_vec_basis]

W_perp = W.complement()

print(W)
print(W_perp)

g = W_perp.basis()
p = W.dimension()
q = W_perp.dimension()

Vector space of degree 1140 and dimension 36 over Rational Field
Basis matrix:
36 x 1140 dense matrix over Rational Field
Vector space of degree 1140 and dimension 1104 over Rational Field
Basis matrix:
1104 x 1140 dense matrix over Rational Field


In [113]:
# action of basis matrix E_ab in glN onto homogeneous polynomial f[i]
def E_action(a, b, i):
    sum = -1*indeterminates[b]*grad_f[i][a]
    return R(sum)

We note `M_f` $:= \tilde{M}_f$

In [134]:
# Pre-processing memoization of entries of M_f

# E_action_f[l][i][j] = (E_ij)(f_l)
E_action_f = [[[sparsify(vp_iso.polynomial_to_vector(E_action(i,j,l))) for j in range(N)] for i in range(N)] for l in range(p)]

# convert g to sparse representation
g_sparse = [sparsify(g[i]) for i in range(q)]

In [155]:
# Construct M_f
M_f = zero_matrix(QQ, p*q, N^2)
for l in range(p):
    for k in range(q):
        for i in range(N):
            for j in range(N):
                M_f[(q*l) + k,(N*i) + j] = sparse_inner_product(E_action_f[l][i][j], g_sparse[k])         

In [156]:
def X_action(X, f):
    grad_test = f.gradient()
    sum = 0
    for a in range(N):
        for b in range(N):
            sum += -X[N*a + b]*indeterminates[b]*grad_test[a]
    return R(sum)

In [157]:
print(M_f.ncols())

print('rank(M_f) =', M_f.rank())
print('dim(ker(M_f)) =', glN.dimension() - M_f.rank())

324
rank(M_f) = 304
dim(ker(M_f)) = 20


In [158]:
print((2^2-1) + (3^2-1) + (3^2-1) +1)

20
